# Binance Live Trading Code
This is an incredibly simple, yet theoretically effective live code for trading a given cryptocurrency pair. The purpose of this code is to illustrate the work that goes into developing an effective and highly robust live code. The actually trading logic is one line of code, however the entire file is more than much longer due to the additional features which support live trading.

## Strategy and Pitfal
if(spread >= cost of trading): trade  
The strategy is aptly named Spread Abuser. It is a HFT code even when run at slow time steps (eg. 1, 2, 5 min). If you backtest this strategy it will often give *theoretical* returns over 100% on many currency pairs.  

The major pitfal of this strategy (and if there wasn't one I wouldn't post the code) is that it fails to move volume effectively. Most orders it puts out will not fulfill and the ones that do are usually the losing trades (margins are tight here). Cryptocurrency markets are relatively illiquid compared to tradition stock markets and there may come a day when that is not the case, however more liquidity most likely will come with smaller spreads which make this strategy more ineffective.

### Imports
All the imports and fetching the date

In [1]:
import json
import os
import time
import numpy as np
import pandas as pd
import statistics as st
import datetime
import smtplib
from email.mime.text import MIMEText
import traceback
from binance.client import Client

np.set_printoptions(threshold=np.nan)

now = datetime.datetime.now()
year = now.year
if(now.month < 10):
    month = '0' + str(now.month)
else:
    month = now.month
if(now.day < 10):
    day = '0' + str(now.day)
else:
    day = now.day

### Settings
Here we define all the parameters necessary. It is the only part of the code that is actively edited before uploading to AWS.

In [2]:
# Settings ---------- Settings ---------- Settings ---------- Settings ---------- Settings ----------

client = Client('put your API','keys here') # there are more secure options than pasting them here directly

baseCurrency = 'ABC' # currency you want to make a return on (for most people will be USD or USDT)
baseCurrency_lower = baseCurrency.lower()

FXcurrency = 'XYZ' # currency/asset you want to game
FXcurrency_lower = FXcurrency.lower()

currencyPair = FXcurrency + baseCurrency

USD_conversion = 1000 # value of ABC in USD

spreadFactor = 1/7 # the strategy uses this constant to price bids/asks effectively (tested imperically, real value is NOT 1/7)

commission = 0.075 # current commission rate on Binance (would recommend using a higher value for a safety margin)

flag = 'Buy' # assumes you start with coin ABC and are looking to use them to buy XYZ

Balance = 25 # balance in terms of FX currency you want to buy/sell
originalBalance = Balance

compounded = 0 # a big advantage of HFT is near continuous compounding, this always is initalized to 0

total_trades = 0

losing_trades = 0

strikes = 0

termination = 0

interval = 150 # 150 seconds = 2.5 minutes between data points

spacing = 2 # to avoid trading anomaly signals we take two data points 2 seconds apart and average their values

delay = 0.5 # time it takes to go through one trading loop

recordFile = '/path/data_{}-{}_{}_{}_{}.txt'.format(baseCurrency_lower, FXcurrency_lower, year, month, day) # in this file we will record everything that happens during runtime, the file name will format to include the currency pair and date started

instructionsFile = '/path/instructions.txt' # this is a file which provides instructions to the live code to stop running or change parameters

myFile = open(recordFile, 'w') # write the recordFile

recipients = ['you@cryptoTrading.com'] # add the recipients of the email updates

market_info = client.get_exchange_info() # each currency pair has differnt limits in regards to decimal precision of prices and quantities

for i in range(0, len(market_info['symbols'])): # this loop quickly finds the coin pair's decimal precision
    checkPair = market_info['symbols'][i]['symbol']
    if(checkPair == currencyPair):
        BalanceRound = float(market_info['symbols'][i]['filters'][1]['stepSize'])
        priceRound = float(market_info['symbols'][i]['filters'][0]['tickSize'])
        BalanceRound = int(abs(np.log10((BalanceRound))))
        priceRound = int(abs(np.log10((priceRound))))

if(('BalanceRound' not in locals()) or ('priceRound' not in locals())):
    print('There\'s a problem finding the tick/step Sizes') # just in case it doesn't find it (never has happened)
else:
    del market_info # clear memorory of large variable we will not use again

### Functions
Here we define all the functions used. There is a function for every REST API call and they are all formatted the same way. I will be the first to admit that the nested Try/Excepts are sloppy and could be done better with a while loop. However, this method allows us to keep track of failed API calls without defining a counter and if statement via the direct prints. When this project was first started it was important for us to keep track of API call success, but now that our error rate is < 0.0112% I have no excuse to not rewrite it nicely.

#### API Methods
Some will be quick to note that I use Binance direct API. Yes I know I could use [ccxt](https://github.com/ccxt/ccxt), I just didn't know about it when I first wrote these codes and retrofitting is low on my to do list - "don't fix it if it ain't broke". In my notebook about data collecting I also address WebSockets vs REST API.

In [3]:
def PriceVerify(info, currencyPair):
    try:
        try:
            try:
                Ticker = client.get_ticker(symbol = currencyPair)
                Bid0 = float(Ticker['bidPrice'])
                Last0 = float(Ticker['lastPrice'])
                Ask0 = float(Ticker['askPrice'])
                info0 = [Bid0, Last0, Ask0]
            except:
                print('GetTicker Fail 1')
                Ticker = client.get_ticker(symbol = currencyPair)
                Bid0 = float(Ticker['bidPrice'])
                Last0 = float(Ticker['lastPrice'])
                Ask0 = float(Ticker['askPrice'])
                info0 = [Bid0, Last0, Ask0]
        except:
            print('GetTicker Fail 2')
            Ticker = client.get_ticker(symbol = currencyPair)
            Bid0 = float(Ticker['bidPrice'])
            Last0 = float(Ticker['lastPrice'])
            Ask0 = float(Ticker['askPrice'])
            info0 = [Bid0, Last0, Ask0]
    except:
        print('GetTicker Fail 3')
        info0 = info # this is the function/API call used the most so as a last resort we pass the previous data points
    return info0

In [4]:
def BuyOrderVerify(Balance, buyPrice, BalanceRound, priceRound, currencyPair):
    Balance = round(Balance, BalanceRound)
    buyPrice = round(buyPrice, priceRound)
    if(priceRound == 8):
        buyPrice = format(buyPrice, '.8f') # I've had issues using scientific notation with Binance
    try:
        try:
            try:
                orderID = client.order_limit_buy(symbol=currencyPair, quantity=Balance, price=buyPrice)['orderId']
            except:
                traceback.print_exc()	
                time.sleep(1)
                print('First BuyOrder did not go through and a second one was issued')
                orderID = client.order_limit_buy(symbol=currencyPair, quantity=Balance, price=buyPrice)['orderId']
        except:
            traceback.print_exc()
            time.sleep(1)
            print('Second BuyOrder did not go through and a third one was issued')
            orderID = client.order_limit_buy(symbol=currencyPair, quantity=Balance, price=buyPrice)['orderId']
    except:
        traceback.print_exc()
        time.sleep(1)
        buyPrice = float(buyPrice)
        BuyOrderVerify(Balance, buyPrice, BalanceRound, priceRound, currencyPair) # all other functions like this are recursive
    return orderID

In [5]:
def SellOrderVerify(Balance, sellPrice, BalanceRound, priceRound, currencyPair): 
    Balance = round(Balance, BalanceRound)
    sellPrice = round(sellPrice, priceRound)
    if(priceRound == 8):
        sellPrice = format(sellPrice, '.8f')
    try:
        try:
            try:
                orderID = client.order_limit_sell(symbol=currencyPair, quantity=Balance, price=sellPrice)['orderId']
            except:
                traceback.print_exc()
                time.sleep(1)
                print('First SellOrder did not go through and a second one was issued')
                orderID = client.order_limit_sell(symbol=currencyPair, quantity=Balance, price=sellPrice)['orderId']
        except:
            traceback.print_exc()
            time.sleep(1)
            print('Second SellOrder did not go through and a third one was issued')
            orderID = client.order_limit_sell(symbol=currencyPair, quantity=Balance, price=sellPrice)['orderId']
    except:
        traceback.print_exc()
        time.sleep(1)
        sellPrice = float(sellPrice)
        SellOrderVerify(Balance, sellPrice, BalanceRound, priceRound, currencyPair)
    return orderID 

In [6]:
def GetOrderStatusVerify(transID, currencyPair): 
    try:
        try:
            try:
                status = client.get_order(symbol=currencyPair, orderId=transID)['status']
            except:
                print('First GetOrderStatus did not go through and a second one was issued')
                status = client.get_order(symbol=currencyPair, orderId=transID)['status']
        except:
            print('Second GetOrderStatus did not go through and a third one was issued')
            status = client.get_order(symbol=currencyPair, orderId=transID)['status']
    except:
        GetOrderStatusVerify(transID, currencyPair)
    return status 

In [7]:
def GetOrderExecutedVerify(transID, currencyPair): # this function replaced GetBalaceVerify because it allows us to have two strategies trading the same currency pair simultaneously
    try:
        try:
            try:
                executedQty = client.get_order(symbol=currencyPair, orderId=transID)['executedQty']
            except:
                executedQty = client.get_order(symbol=currencyPair, orderId=transID)['executedQty']
                print('First GetOrderExecuted did not go through and a second one was issued')
        except:
            executedQty = client.get_order(symbol=currencyPair, orderId=transID)['executedQty']
            print('Second GetOrderExecuted did not go through and a third one was issued')
    except:
        GetOrderExecutedVerify(transID, currencyPair)
    return float(executedQty)

In [8]:
def CancelVerify(transID, currencyPair):
    try:
        try:
            try:
                cancel = client.cancel_order(symbol=currencyPair, orderId=transID)
            except:
                print('First cancel did not go through and a second one was issued')
                cancel = client.cancel_order(symbol=currencyPair, orderId=transID)
        except:
            print('Second cancel did not go through and a third one was issued')
            cancel = client.cancel_order(symbol=currencyPair, orderId=transID)
    except:
        CancelVerify(transID, currencyPair)
    return cancel

In [9]:
def GetBalanceVerify(FXcurrency): # we don't use this function anymore, but keep it around in case
    try:
        try:
            try:
                quantity = client.get_asset_balance(asset=FXcurrency)['free']
            except:
                print('First GetBalance did not go through and a second one was issued')
                quantity = client.get_asset_balance(asset=FXcurrency)['free']
        except:
            print('Second GetBalance did not go through and a third one was issued')
            quantity = client.get_asset_balance(asset=FXcurrency)['free']
    except:
        GetBalanceVerify(FXcurrency)
    return float(quantity)

In [10]:
def USD_PriceVerify(Last, baseCurrency): # convert return in your base currency to USD
    if(baseCurrency != 'USDT'):
        USD_pair = baseCurrency + 'USDT'
        try:
            try:
                try:
                    Ticker = client.get_ticker(symbol=USD_pair)
                    Last0 = Ticker['lastPrice']	
                except:
                    Ticker = client.get_ticker(symbol=USD_pair)
                    Last0 = Ticker['lastPrice']
                    print('GetTicker Fail 1')
            except:
                Ticker = client.get_ticker(symbol=USD_pair)
                Last0 = Ticker['lastPrice']
                print('GetTicker Fail 2')
        except:
            Last0 = Last
            print('GetTicker Fail 3')
        return float(Last0)
    else:
        return 1 # if your base currency is USD your conversion rate is 1

In [11]:
def EmailUpdate(recipients, msg): # function to send email updates about performance
    server = smtplib.SMTP('smtp.server.com', 465) # this will vary, find server/port for your email host
    server.ehlo()
    server.starttls()
    server.login('you@cryptoTrading.com', 'password') # once again only put your password here if you want to get hacked

    server.sendmail('you@cryptoTrading.com', recipients, msg.as_string())
    server.quit()

### Start-Up
Since this strategy only evaluates one data point at a time the startup is just grabbing one data point. All the live tradin in in a while loop within a try statement. This is because at the end there is an except to notify us by email if an error occurs, what it is, our current P/L is, and if our position if Buy (assets in base currency) or Sell (assets in FX currency).

In [12]:
try:
    # Start-Up ---------- Start-Up ---------- Start-Up ---------- Start-Up ---------- Start-Up ----------
    timer = 1

    Ticker = client.get_ticker(symbol = currencyPair) # the only naked API call, if it doesn't error here it probably never will
    Bid = Ticker['bidPrice']
    Last = Ticker['lastPrice']
    Ask = Ticker['askPrice']
    info = [Bid, Last, Ask]


### Alternative Start-Up
In other codes that need to see a large sample of data at once to make a decision one can start the code and let it collect data until it has a sufficient number of data points. Eg. if you need 50 data points at a 4 minute interval this means you have a 2 hour 40 minute wait before the code begins trading. The more effective method is to pull the number of data points from our CSV of recorded data and then begin trading immediately.  

length_needed would be the number of most recent data points to pull. Without proper time syncronization it is likely that the first data point collected in this code is then not exactly 4 minutes after the most recent data point pulled from the CSV. The smaller the time step the more insignificant this error becomes or you could just timestamp your data and make the live code synchonize. Either way after collecting the first 50 data points (or however many you need) all the data in the live code will be collected by itself.

In [13]:
    Ticker = client.get_ticker(symbol=currencyPair)
    Bid = Ticker['bidPrice']
    Last = Ticker['lastPrice']
    Ask = Ticker['askPrice']

    grabinfo_data = pd.read_csv(grabinfoFile, header=1) # grabinfoFile would then be defined in the Settings

    bids = grabinfo_data['bidPrice']
    lasts = grabinfo_data['lastPrice']
    asks = grabinfo_data['askPrice']

    bids = bids[len(bids) - int(lenth_needed):len(bids)]
    lasts = lasts[len(lasts) - int(lenth_needed):len(lasts)]
    asks = asks[len(asks) - int(lenth_needed):len(asks)]

    allPrices = [asks, lasts, bids]
    allPrices = np.array(allPrices)

    del bids, lasts, asks, grabinfo_data # clear memory of variables we no longer need

### Trading
This is best part! The last line in the section below is the single line of logic begind this entire strategy, everything else is just support!

In [14]:
# Trading ---------- Trading ---------- Trading ---------- Trading ---------- Trading ----------

    transactionList = []
    commissionList = []
    append = True # this will be explained below

    while True:
        info1 = PriceVerify(info, currencyPair) # get data points 1

        time.sleep(spacing) # hold a second (or two)

        info2 = PriceVerify(info, currencyPair) # get data points 2
        Bid2, Last2, Ask2 = info2[0], info2[1], info2[2] # only use the second ones for pricing

        info = np.zeros(3)
        for i in range(0, 3):
            info[i] = np.mean([info1[i], info2[i]]) # define the average to use for calculations

        Bid, Last, Ask = info[0], info[1], info[2]

        margin = Ask/(1 + commission/100) - Bid*(1 + commission/100) # margin is spread - cost of trading

### Buying
Here is the if statement for buying. It's not that complicated... except in the case that your order doesn't go through. Try to appreciate how little of it is actually trading versus keeping track of ordeers, volume moved, recording all of this, etc.

In [15]:
        if((margin >= 0) and (flag == 'Buy' or flag == 'cancelBuy')):

            myFile = open(recordFile, 'a')
            with myFile:
                myFile.write('Buying {}\n'.format(FXcurrency))
            print('Buying {}'.format(FXcurrency))

            Balance += compounded
            if(Balance >= originalBalance*2): # in the case we do 100% it might become tought to move volume, so we reset the balance
                Balance = originalBalance
                myFile = open(recordFile, 'a')
                with myFile:
                    myFile.write('The code has reached a 100% return, so the balance has been reset\n')
                print('The code has reached a 100% return, so the balance has been reset') # never seen this print, but I like to be ambitious

            buyPrice = Bid2 + spreadFactor*(Ask2 - Bid2)
            if(append == True):
                transactionList.append(-buyPrice)
                commissionList.append(1 + commission/100)
            else:
                transactionList.pop()
                transactionList.append(-buyPrice)

            transID = BuyOrderVerify(Balance, buyPrice, BalanceRound, priceRound, currencyPair) # in a perfect world this would be the only line you need

            myFile = open(recordFile, 'a') # record the data in the terminal and in the recordFile
            with myFile:
                myFile.write('Quantity @ Price\n')
                myFile.write('{} @ {}\n'.format(Balance, buyPrice))
            print('Quantity @ Price')
            print('{} @ {}'.format(Balance, buyPrice))

#### Cancelling a Buy Order - HFT
Read the comments in the code to follow the logic. This would be the buy cancel loop for this specific code, but in the next section we will look at an alternative one. 

In [16]:
            time.sleep(interval - spacing - delay) # wait for the time interval to end
            status = GetOrderStatusVerify(transID, currencyPair) # did we move volume?
            if(status != 'FILLED'): # Ideally, yes. Realistically, no
                myFile = open(recordFile, 'a')
                with myFile:
                    myFile.write('Canceling Buying {}\n'.format(FXcurrency))
                print('Canceling Buying {}'.format(FXcurrency))
                CancelVerify(transID, currencyPair) # cancel the order

                amountBought = GetOrderExecutedVerify(transID, currencyPair) # check how much we bought
                if(amountBought >= 0.75*Balance): # more than 75% quantity fulfilled?
                    sellBalance = amountBought
                    flag = 'Sell' # make peace with it and move on
                    append = True # append is set to True meaning we will record the next transactions
                else:
                    if(flag == 'Buy'): # or else you're going to have to buy more
                        sellBalance = amountBought # but we also have to keep track of how much we're going to sell
                    else:
                        sellBalance += amountBought
                    flag = 'cancelBuy'
                    append = False # append is set to False because we could not fulfill the order, so the price must have moved up and is less optimal, so we will overwrite the previously recorded value with the new higher and undesirable price in order to make profit calculations conservative
            else: # ideally though we just come out here everytime 
                sellBalance = Balance
                flag = 'Sell'
                append = True

#### Cancelling a Buy Order - Alternative (Not HFT)
The code above is unique to this strategy because we want to buy at very specific point at an even more specific price and then sell minutes later. For this reason if we do not fulfill our order the code reroutes itself through the buy loop again ensuring that the buy condition is triggered again.  

In other more slow trading codes, positions will be held for 6-24 hours, sometimes even longer. In these cases the margins on buying and selling or not as important since the code is trying to capture gains or larger price movments. Therefore the cancel loop is written so that once the buy trigger is met we will continue to put out buy orders until the volume is fulfilled even if we encounter price slippage. To combat priced slippage though, the time interval is then adjusted.  

Reading throuh the code and comments is the best way to follow how it works. This would follow the code in the Buying Section.

In [17]:
            flag = 'Sell' # not this cancel loop begins by assuming we have fulfilled the order

            time.sleep(interval - spacing - delay) # wait for the time interval to end
            status = GetOrderStatusVerify(transID, currencyPair) # did we move volume?
            if(status != 'FILLED'): # if FILLED all is well and move on
                myFile = open(recordFile, 'a')
                with myFile:
                    myFile.write('Canceling Buying {}\n'.format(FXcurrency))
                print('Canceling Buying {}'.format(FXcurrency))
                CancelVerify(transID, currencyPair) # if not cancel the order
                flag = 'cancelBuy' # switch the status to cancelBuy
                amountBought = GetOrderExecutedVerify(transID, currencyPair)
                cancelBalance = Balance - amountBought # calculate how much more we have to buy
                interval = interval/5 # divide the interval by 5 so that we go through the next loop faster and can issue our next order quickly


        elif(flag == 'cancelBuy'): # the new elif statement only requires the flag be cancelBuy, our buy trigger is most likely no longer met, but we want to make sure we lock in our position regardless as price slippage in not that important here
            myFile = open(recordFile, 'a')
            with myFile:
                myFile.write('Buying {} after a cancel\n'.format(FXcurrency))
            print('Buying {} after a cancel'.format(FXcurrency))

            interval = interval*5 # restore the interval

            buyPrice = Bid2 + spreadFactor*(Ask2 - Bid2)
            transactionList.pop()
            transactionList.append(-buyPrice) # Obviously the price has moved up from where we placed or order, otherwise our order would have completed. So in the transaction list we overwrite the buy price with the new higher one so that our profit calculations are conservative using the higher values

            transID = BuyOrderVerify(cancelBalance, buyPrice, BalanceRound, priceRound, currencyPair)

            flag = 'Sell' # assume we fulfilled the order

            time.sleep(interval - spacing - delay)
            status = GetOrderStatusVerify(transID, currencyPair)
            if(status != 'FILLED'): 
                myFile = open(recordFile, 'a')
                with myFile:
                    myFile.write('Canceling Buying {}\n'.format(FXcurrency))
                print('Canceling Buying {}'.format(FXcurrency))
                CancelVerify(transID, currencyPair) # if not cancel the order, change the flag, adjust the interval, and the next iteration will come through here again until we obtain the quantity we desire
                flag = 'cancelBuy'
                amountBought = GetOrderExecutedVerify(transID, currencyPair)
                cancelBalance = cancelBalance - amountBought
                interval = interval/5

### Selling
Here we go through the elif sell loop. It ressembles the buy loop, except many things are simply reversed. One key point is that there is far more performance/data logging in this loop as here we liquidated everything back to our base currency and can check our P/L.  

For simplicity the sell cancel loops are omitted. They can easily be derived from the buy cancel loops if you would like to incorporate them into your own code.

In [18]:
        elif((margin >= 0) and (flag == 'Sell')):
            myFile = open(recordFile, 'a')
            with myFile:
                myFile.write('Selling {}\n'.format(FXcurrency))
            print('Selling {}'.format(FXcurrency))

            sellPrice = Ask2 - spreadFactor*(Ask2 - Bid2)

            if(append == True):
                transactionList.append(sellPrice)
                commissionList.append(1 - commission/100)
            else:
                transactionList.pop()
                transactionList.append(sellPrice)

            transID = SellOrderVerify(Balance, sellPrice, BalanceRound, priceRound, currencyPair)

            profit = sum(transactionList*np.transpose(commissionList))*originalBalance # calculate profit on base currency accounting for transaction costs
            profit = round(profit, 5)
            USD_conversion = USD_PriceVerify(USD_conversion, baseCurrency)
            USDprofit = round(profit*USD_conversion, 2)

            if(buyPrice >= sellPrice): # this is obviously bad
                losing_trades += 1 
            if(buyPrice >= sellPrice or profit <= 0):
                strikes += 1 # so we penalize it
                if(strikes == 3): # 3 strikes and you're out! just like baseball... jk, please don't use 3, find your own risk threshold
                    myFile = open(recordFile, 'a')
                    with myFile:
                        myFile.write('Code lost money 3 times in a row\n')
                    print('Code lost money 3 times in a row')
                    termination = 1 # so we stop trading
            else:
                strikes = 0 # if we make a winning trade we reset the strikes

            total_trades += 1 # keep track of number of trades

            winPct = round((1 - losing_trades/total_trades)*100, 2) # percentage of trades that made money
            compounded = (Balance/Last)*(sellPrice*(1 - commission/100) - buyPrice*(1 + commission/100)) # calculate the amount to compounded (can be negative if trades are losing money)
            returnPct = round(Balance/originalBalance - 1, 2) # percentage return (relative to base currency)
            days_live = round(timer*interval/3600/24, 1) # how long has the code been running live

            if(total_trades%5 == 0): # every 5 trades send an email update 
                msg = MIMEText('Transaction list = {}\nTotal trades = {}\nWinning trade percentage = {}%\nProfit = {} {}\nProfit ~= $ {}\nReturn = {}%\nRunning live for {} days'.format(transactionList, total_trades, winPct, profit, baseCurrency, USDprofit, returnPct, days_live))
                # the message includes the transaction list, number of total total trades, winning trade percentage, profit in base currency, profit in USD, return percentage relative to base currency, and days_live
                if(profit > 0): # fun little if statement to add the important details to the email subject line
                    msg['Subject'] = 'PRACTICE Spread Abuser {}-{} Code - Good News'.format(baseCurrency, FXcurrency)
                else:
                    msg['Subject'] = 'PRACTICE Spread Abuser {}-{} Code - Bad News'.format(baseCurrency, FXcurrency)

                EmailUpdate(recipients, msg) # send it

            myFile = open(recordFile, 'a') # record all of this in the terminal and in the recordFile
            with myFile: # can't stress enough the value of having this everywhere - the goal should be to never have to manually log into Binance ever again to check the status of your account
                myFile.write('Quantity @ Price\n')
                myFile.write('{} @ {}\n'.format(Balance, sellPrice))
                myFile.write('Transaction List = {}\n'.format(transactionList))
                myFile.write('Total trades = {}\n'.format(total_trades))
                myFile.write('Winning trade percentage = {}%\n'.format(winPct))
                myFile.write('Profit = {} {}\n'.format(profit, baseCurrency))
                myFile.write('Profit ~= $ {}\n'.format(USDprofit))
                myFile.write('Return = {}%\n'.format(returnPct))
            print('Quantity @ Price')
            print('{} @ {}'.format(Balance, sellPrice))
            print('Transaction List = {}'.format(transactionList))
            print('Total trades = {}'.format(total_trades))
            print('Winning trade percentage = {}%'.format(winPct))
            print('Profit = {} {}'.format(profit, baseCurrency))
            print('Profit ~= $ {}'.format(USDprofit))
            print('Return = {}%'.format(returnPct))

            flag = 'Buy'

            time.sleep(interval - spacing - delay)

### Else
Most of the time, with the exception of HFT strategies, the code does not trigger a buy or sell. Thus it comes through the final else statement, which is where the code spends the large amjority of it's runtime.

In [19]:
        else:
            time.sleep(interval - spacing - delay)

### Termination
Unfortunately, sometimes the strategy loses money and termination is triggered. The if statement also includes that the flag is not cancelSell as this ensure that when the code stops running all funds are in the base currency. Otherwise the code may stop running even though an order is not fulfilled and you will have to resolve it manually on the exchange. Finally, the code breaks out of the trading loop.

In [20]:
        if(termination == 1 and flag != 'cancelSell'):
            myFile = open(recordFile, 'a')
            with myFile:
                myFile.write('Breaking out of trading loop\n')
            print('Breaking out of trading loop')

            msg = MIMEText('The Spread Abuser {}-{} Code lost money and stopped trading\nTransaction list = {}\nTotal trades = {}\nWinning trade percentage = {}%\nProfit = {} {}\nProfit ~= $ {}\nReturn = {}%'.format(baseCurrency, FXcurrency, transactionList, total_trades, winPct, profit, baseCurrency, USDprofit, returnPct))
            msg['Subject'] = 'Spread Abuser {}-{} Code - Losing Money'.format(baseCurrency, FXcurrency)

            EmailUpdate(recipients, msg)
            break

### Instructions
Sometimes you make improvements to a strategy and want to update the live code. Other times human intuition or knowledge of external factors, things the code won't consider, make you decide to take down a code before a coming market downturn. You could log into AWS and hit control C, but a more elagant and versatile way is to read from the instructions file.  

This has the same feature as the termination whereby even if the instructions are to stop running the code waits until all funds are back in the base currency. Elif statements can easily be added for other conditions. For example this strategy's most influential input is the commission rate, but for strategies that take many hyperparameters these could be manually adjusted and then fed into the live code to be updated here.

In [21]:
        try:
            instructions = open(instructionsFile, 'r')
            instructions = instructions.read()
            if(instructions == 'stop' and flag == 'Buy'):
                msg = MIMEText('The code was taken down for maintenance\n\nTotal Runtime Performance\n\nTransaction list = {}\nTotal trades = {}\nWinning trade percentage = {}%\nProfit = {} {}\nProfit ~= $ {}\nReturn = {}%\nRan live for {} days'.format(transactionList, total_trades, winPct, profit, baseCurrency, USDprofit, returnPct, days_live))
                msg['Subject'] = 'Spread Abuser {}-{} Code - Maintenance'.format(baseCurrency, FXcurrency)

                EmailUpdate(recipients, msg)

                myFile = open(recordFile, 'a')
                with myFile:
                    myFile.write('The code was taken down for maintenance\n')
                print('The code was taken down for maintenance')
                break
        except:
            pass

### Parameter Resets
While manual parameter overwriting via txt instruction file could easily be incorporated into the above code, we normally build this feature directly into the strategy. Most of our successful strategies (which will never see the light of day on GitHub) take several key parameter inputs. We rigorously backtest and optimize to find these and hard code them into our strategies. There are times though where it is more favorable to let the code auto update them.  

In this case a variable called reset and a function called Parameter_Chooser would be previously defined. Imagine reset is equal to a weeks worth of data points, then the code will update twice a week (every 3.5 days) using a weeks worth of data to backtest. This will give a variable named updatedParameters, which will give has a lot of information in addition to the parameters, such as return, return percentage, number of trades, and winning trade percentage. From this we select the parameters as a variable named chosenParameters, which would overwrite chosenParameters defined in the Settings. Both are printed and recorded so that we can see the parameters as well as their performance in backtesting.

In [22]:
        if(((timer%reset == reset/2) and (timer >= reset)) or (timer%reset == 0)):
            backtestData = data.loc[len(data)-reset:len(data), :] # written for a pd.DataFrame

            updatedParameters = Parameter_Chooser(a, b, c, d, e, f, commission, h, backtestData)
            chosenParameters = updatedParameters[1:5]
            myFile = open(recordFile, 'a')
            with myFile
                myFile.write('Updated Parameters\n')
                myFile.write(str(updatedParameters)+'\n')
                myFile.write(str(chosenParameters)+'\n')
            print('Updated Parameters')
            print(updatedParameters)
            print(chosenParameters)

### Clearing Memory
Fortunately this strategy only needs to store one data point at a time. Most other strategies need to store much larger amounts of data and if many codes (sometimes we run two different strategies over several currency pairs all simultaneously) are running at once this could begin to use up AWS memory. Most actual strategies we use tend to require about 50-250 data points at a time and perhaps several thousand if auto updating is included. Ever so often we trim the data to make sure DataFrames, Series, and arrays don't grow indefinitely. Below is sample code for this feature.

In [23]:
        if(len(data) >= 10000):
                data = data.loc[len(data)-10000:len(data), :] # written for a pd.DataFrame

### Increment
Increment the timer and record it

In [24]:
        timer += 1
            myFile = open(recordFile, 'a')
            with myFile:
                myFile.write('{}\n'.format(timer))
            print(timer)

### Errors
Lastly is the final except in case of an error. The code can error in the start up when the user hits Enter, but this would be caught immediately. Errors that happen while trading are very important to be notified of because it is too cumbersome to manually check record files or consoles directly.  

Since cryptocurrency markets are 24/7 it is also useful to get a brief performance log in this email. This way if we receive this email on a Friday night and see that the profit is positive and flag is Buy we know it's not urgent, wheras profit is negative and flag is Sell would be higher priority. 

In [25]:
except:
	traceback.print_exc()

	days_live = round(timer*interval/3600/24, 1)

	msg = MIMEText('The {}-{} Code errored out and needs to be checked\n\nThe current profit is {}\n\nThe current position is {}\n\nRan live for {} days'.format(baseCurrency, FXcurrency, profit, flag, days_live))
	msg['Subject'] = 'RSI {}-{} Code - Error'.format(baseCurrency, FXcurrency)

	EmailUpdate(recipients, msg)